In [1]:
import boto3
from kmsgm import KMSManager

kms_client = boto3.client('kms')
kms_manager = KMSManager(kms_client)

In [2]:
#kms_manager.create_key('alias/my-key')
kms_manager.select_key("f5d74a0b-31b8-418e-b50e-d3a99c608f54")

Key f5d74a0b-31b8-418e-b50e-d3a99c608f54 selected.
Key f5d74a0b-31b8-418e-b50e-d3a99c608f54 is already Disabled.


In [3]:
kms_manager.create_alias("alias/my-key")

Created alias alias/my-key for key f5d74a0b-31b8-418e-b50e-d3a99c608f54.


In [4]:
kms_manager.describe_key()

Got key f5d74a0b-31b8-418e-b50e-d3a99c608f54:
{'AWSAccountId': '089715336747',
 'Arn': 'arn:aws:kms:us-east-1:089715336747:key/f5d74a0b-31b8-418e-b50e-d3a99c608f54',
 'CreationDate': datetime.datetime(2024, 3, 5, 16, 4, 13, 685000, tzinfo=tzlocal()),
 'CustomerMasterKeySpec': 'SYMMETRIC_DEFAULT',
 'Description': 'alias/my-key',
 'Enabled': True,
 'EncryptionAlgorithms': ['SYMMETRIC_DEFAULT'],
 'KeyId': 'f5d74a0b-31b8-418e-b50e-d3a99c608f54',
 'KeyManager': 'CUSTOMER',
 'KeySpec': 'SYMMETRIC_DEFAULT',
 'KeyState': 'Enabled',
 'KeyUsage': 'ENCRYPT_DECRYPT',
 'MultiRegion': False,
 'Origin': 'AWS_KMS'}


In [4]:
kms_manager.actual_state = "Enabled"

Key f5d74a0b-31b8-418e-b50e-d3a99c608f54 set to Enabled.


In [5]:
kms_manager.actual_state

'Enabled'

In [8]:
encrpyt_text = kms_manager.encrypt("Hello World")
print(encrpyt_text)

b'\x01\x02\x02\x00xN;\x98\xe5$C\xd2\xb12Z\xb2\xcf\xeb\xcf\xc1\xf3\xdc\x0c\xeb\xea^\xdbq6\x84^\xbc\xe7\x93\xe7>\x1e\x01\x96\xc9Kw\xb1\xdf\xa8\x93u\xfat\x9e\xda\xd1\x9b\xd3\x00\x00\x00i0g\x06\t*\x86H\x86\xf7\r\x01\x07\x06\xa0Z0X\x02\x01\x000S\x06\t*\x86H\x86\xf7\r\x01\x07\x010\x1e\x06\t`\x86H\x01e\x03\x04\x01.0\x11\x04\x0c\xb526#\x02\x86H\xea\xce?\xd4;\x02\x01\x10\x80&\xc9\xc1O\xa9\t([r\xfa\xf8\x13\x89\xc5\xc8!\xf2\x97\xc0\xeaj$\x94~\xd5\xf3\rB\x1csb".\x81\xe2\x8b\xb5\xcc\x88'


In [9]:
decrypt_text = kms_manager.decrypt(encrpyt_text)
print(decrypt_text)	

Hello World
